In [8]:
!git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null
!git clone https://github.com/chinmusique/outcome-prediction.git &> /dev/null
from seminaire.evaluation.jour2.listes.listes import 

ImportError: ignored

# 1 Introduction

In my last publication, I started the post series on the topic of text pre-processing. In it, I first covered all the possible applications of Text Cleaning.

Now I will continue with the topics Tokenization and Stop Words.

For this publication the processed dataset Amazon Unlocked Mobile from the statistic platform “Kaggle” was used as well as the created Example String. You can download both files from my “GitHub Repository”.

# Import the Libraries and the Data


In [42]:
import pandas as pd
import numpy as np

import pickle as pk

import warnings
warnings.filterwarnings("ignore")


from bs4 import BeautifulSoup
import unicodedata
import re

import nltk

nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords


from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from wordcloud import WordCloud

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [31]:
import json
import pandas as pd
from pathlib import Path

!git clone https://github.com/chinmusique/outcome-prediction.git

# creating list of json files with full paths
paths = Path(r'/content/outcome-prediction/data/manual_annotation').glob("*.json")

df_list = [] # empty list to store dataframes during the loop
for p in paths:
    data = json.load(open(p))
    data = data["annotations"]
    df_nested_list = pd.json_normalize(
        data,
        meta=['name','description','custom_fields'],)
    df_nested_list['file_name'] = p.name  # creating column to store file name: p.name
    df_list.append(df_nested_list) # dataframe to the list

df_raw = pd.concat(df_list, axis=0, ignore_index=True) # creating One Large Dataframe from all stored in the list
print(df_raw.columns)

df = df_raw.drop(columns=["role",	"class",	"index",	"contentIdx.start",	"contentIdx.end", "_id", "index"])
print(df.columns)

fatal: destination path 'outcome-prediction' already exists and is not an empty directory.
Index(['_id', 'role', 'content', 'class', 'index', 'contentIdx.start',
       'contentIdx.end', 'file_name'],
      dtype='object')
Index(['content', 'file_name'], dtype='object')


In [32]:
df.head()


,content,file_name
0,We reverse the orders of the trial court denyi...,77129.json
1,We therefore reverse Defendant's conviction an...,77129.json
2,NaN,77129.json
3,"For the reasons discussed herein, we reverse ...",18357.json
4,"Accordingly, the cause is reversed and remande...",18357.json


3 Definition of required Functions


All functions are summarized here. I will show them again where they are used during this post if they are new and have not been explained yet.



In [36]:
def word_count_func(text):
    '''
    Counts words within a string
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Number of words within a string, integer
    ''' 
    return len(text.split())

In [37]:
def remove_english_stopwords_func(text):
    '''
    Removes Stop Words (also capitalized) from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without Stop Words
    ''' 
    # check in lowercase 
    t = [token for token in text if token.lower() not in stopwords.words("english")]
    text = ' '.join(t)    
    return text

## 4 Text Pre-Processing


### 4.1 (Text Cleaning)


I have already described this part in the previous post. See here: Text Cleaning

### 4.2 Tokenization


Tokenisation is a technique for breaking down a piece of text into small units, called tokens. A token may be a word, part of a word or just characters like punctuation.

Tokenisation can therefore be roughly divided into three groups:

Word Tokenization
Character Tokenization and
Partial Word Tokenization (n-gram characters)
In the following I will present two tokenizers:

Word Tokenizer
Sentence Tokenizer
Of course there are some more. Find the one on the NLTK Homepage which fits best to your data or to your problem solution.



In [38]:
text_for_tokenization = \
"Hi my name is Michael. \
I am an enthusiastic Data Scientist. \
Currently I am working on a post about NLP, more specifically about the Pre-Processing Steps."

text_for_tokenization

'Hi my name is Michael. I am an enthusiastic Data Scientist. Currently I am working on a post about NLP, more specifically about the Pre-Processing Steps.'

### 4.2.1 Word Tokenizer


To break a sentence into words, the word_tokenize() function can be used. Based on this, further text cleaning steps can be taken such as removing stop words or normalising text blocks. In addition, machine learning models need numerical data to be trained and make predictions. Again, tokenisation of words is a crucial part of converting text into numerical data.



In [43]:
words = word_tokenize(text_for_tokenization)
print(words)

['Hi', 'my', 'name', 'is', 'Michael', '.', 'I', 'am', 'an', 'enthusiastic', 'Data', 'Scientist', '.', 'Currently', 'I', 'am', 'working', 'on', 'a', 'post', 'about', 'NLP', ',', 'more', 'specifically', 'about', 'the', 'Pre-Processing', 'Steps', '.']


In [44]:
print('Number of tokens found: ' + str(len(words)))


Number of tokens found: 30


### 4.2.2 Sentence Tokenizer


Now the question arises, why do I actually need to tokenise sentences when I can tokenise individual words?

An example of use would be if you want to count the average number of words per sentence. How can I do that with the Word Tokenizer alone? I can’t, I need both the sent_tokenize() function and the word_tokenize() function to calculate the ratio.

In [45]:
sentences = sent_tokenize(text_for_tokenization)
print(sentences)

['Hi my name is Michael.', 'I am an enthusiastic Data Scientist.', 'Currently I am working on a post about NLP, more specifically about the Pre-Processing Steps.']


In [46]:
print('Number of sentences found: ' + str(len(sentences)))


Number of sentences found: 3


In [47]:
for each_sentence in sentences:
    n_words=word_tokenize(each_sentence)
    print(each_sentence)   

Hi my name is Michael.
I am an enthusiastic Data Scientist.
Currently I am working on a post about NLP, more specifically about the Pre-Processing Steps.


In [48]:
for each_sentence in sentences:
    n_words=word_tokenize(each_sentence)
    print(len(n_words))

6
7
17


### 4.2.3 Application to the Example String


In [53]:
tokens_clean_text = word_tokenize(".".join(df["content"].tolist()))
print(tokens_clean_text)

['We', 'reverse', 'the', 'orders', 'of', 'the', 'trial', 'court', 'denying', 'Defendant', "'s", 'motion', 'to', 'dismiss', 'the', 'indictment', '..', 'We', 'therefore', 'reverse', 'Defendant', "'s", 'conviction', 'and', 'remand', 'the', 'case', 'with', 'instructions', 'to', 'discharge', 'Defendant', '..', 'nan', '.', 'For', 'the', 'reasons', 'discussed', 'herein', ',', 'we', 'reverse', 'and', 'remand', 'for', 'further', 'proceedings', '..', 'Accordingly', ',', 'the', 'cause', 'is', 'reversed', 'and', 'remanded', 'for', 'further', 'proceedings', 'consistent', 'with', 'this', 'opinion', '..', 'nan', '.', 'For', 'the', 'reasons', 'that', 'follow', ',', 'we', 'affirm', '..', 'Based', 'on', 'the', 'foregoing', ',', 'we', 'affirm', 'the', 'district', 'court', "'s", 'judgment', '..', 'nan.We', 'hold', 'that', 'the', 'Department', 'properly', 'considered', 'the', 'applicable', 'regulatory', 'requirements', ',', 'reject', 'the', 'other', 'claims', ',', 'and', 'affirm', '..', 'The', 'decision', 

In [54]:
print('Number of tokens found: ' + str(len(tokens_clean_text)))


Number of tokens found: 30301


4.2.4 Application to the DataFrame


In [55]:
df.head()

,content,file_name,clean_content
0,We reverse the orders of the trial court denyi...,77129.json,We reverse the orders of the trial court denyi...
1,We therefore reverse Defendant's conviction an...,77129.json,We therefore reverse Defendant's conviction an...
2,nan,77129.json,nan
3,"For the reasons discussed herein, we reverse ...",18357.json,"For the reasons discussed herein, we reverse ..."
4,"Accordingly, the cause is reversed and remande...",18357.json,"Accordingly, the cause is reversed and remande..."


Here I set a limit for the column width so that it remains clear. This setting should be reset at the end, otherwise it will remain.



In [56]:
pd.set_option('display.max_colwidth', 30)


In [67]:
df['tokens'] = df['content'].apply(word_tokenize)
df['Word_Count'] = df['content'].apply(word_count_func)


In [65]:
df.head()


,content,file_name,clean_content,tokens,Token_Count,Word_Count
0,We reverse the orders of t...,77129.json,We reverse the orders of t...,"[We, reverse, the, orders,...",17,15
1,We therefore reverse Defen...,77129.json,We therefore reverse Defen...,"[We, therefore, reverse, D...",16,14
2,nan,77129.json,nan,[nan],1,1
3,For the reasons discussed...,18357.json,For the reasons discussed...,"[For, the, reasons, discus...",14,12
4,"Accordingly, the cause is ...",18357.json,"Accordingly, the cause is ...","[Accordingly, ,, the, caus...",16,14


In [68]:
df['Token_Count'] = df['tokens'].str.len()

df.head()

,content,file_name,clean_content,tokens,Token_Count,Word_Count
0,We reverse the orders of t...,77129.json,We reverse the orders of t...,"[We, reverse, the, orders,...",17,15
1,We therefore reverse Defen...,77129.json,We therefore reverse Defen...,"[We, therefore, reverse, D...",16,14
2,nan,77129.json,nan,[nan],1,1
3,For the reasons discussed...,18357.json,For the reasons discussed...,"[For, the, reasons, discus...",14,12
4,"Accordingly, the cause is ...",18357.json,"Accordingly, the cause is ...","[Accordingly, ,, the, caus...",16,14


It is always worthwhile (I have made a habit of doing this) to have the number of remaining words or tokens displayed and also to store them in the data record. The advantage of this is that (especially in later process steps) it is very quick and easy to see what influence the operation has had on the quality of my information. Of course, this can only be done on a random basis, but it is easy to see whether the function applied had negative effects that were not intended. Or you look at a case difference if you don’t know which type of algorithm (for example, in normalisation) fits my data better.



In [69]:
print('Average of words counted: ' + str(df['Word_Count'].mean()))
print('Average of tokens counted: ' + str(df['Token_Count'].mean()))


Average of words counted: 14.843612334801762
Average of tokens counted: 16.868392070484582


Ok interesting, the average number of words has increased slightly. Let’s take a look at what caused that:



In [72]:
df_subset = df[['content', 'Word_Count', 'tokens', 'Token_Count']]
df_subset['Diff'] = df_subset['Token_Count'] - df_subset['Word_Count']


df_subset = df_subset[(df_subset["Diff"] != 0)]
df_subset.sort_values(by='Diff', ascending=False)


,content,Word_Count,tokens,Token_Count,Diff
1740,"Therefore, we reverse and ...",40,"[Therefore, ,, we, reverse...",61,21
886,We uphold the district cou...,43,"[We, uphold, the, district...",61,18
737,Concluding that Defendant ...,61,"[Concluding, that, Defenda...",76,15
1236,We affirm the trial court'...,76,"[We, affirm, the, trial, c...",90,14
1456,We note jurisdiction under...,27,"[We, note, jurisdiction, u...",40,13
...,...,...,...,...,...
1307,We affirm the decision of ...,8,"[We, affirm, the, decision...",9,1
1305,We affirm.,2,"[We, affirm, .]",3,1
1303,We affirm the Court of Ap...,22,"[We, affirm, the, Court, o...",23,1
1302,We now affirm the Court of...,11,"[We, now, affirm, the, Cou...",12,1


Note: In the following I do not take the first row from the sorted dataset, but from the created dataset df_subset.



In [78]:
df_subset['content'].str.split().iloc[0]


['We',
 'reverse',
 'the',
 'orders',
 'of',
 'the',
 'trial',
 'court',
 'denying',
 "Defendant's",
 'motion',
 'to',
 'dismiss',
 'the',
 'indictment.']

In [75]:
df_subset['tokens'].iloc[0]


['We',
 'reverse',
 'the',
 'orders',
 'of',
 'the',
 'trial',
 'court',
 'denying',
 'Defendant',
 "'s",
 'motion',
 'to',
 'dismiss',
 'the',
 'indictment',
 '.']

Here we see the reason: The tokenizer has turned ‘cannot’ into ‘can not’.